In [1]:
import torch
import numpy as np
import openmm
import openmm.app as app
import openmm.unit as u
import bisect
from rdkit.Chem import TorsionFingerprints
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
from rdkit.Geometry import Point3D

import sys
sys.path.append("main")
import pickle
import main.utils

In [2]:
true_pdb = "1uao.pdb"
mol = Chem.rdmolfiles.MolFromPDBFile(true_pdb, removeHs=False)


In [3]:
md_sim = main.utils.MDSimulatorPDB(true_pdb)
md_sim.optimize_confs(mol)
md_sim.get_conformer_energies(mol)

array([ 12.21277936,  16.79118679,  12.63188958,  18.63213103,
        27.10002818,  16.85683164,  -1.56992294,  48.54843964,
        20.65753476, -19.34282877,  30.03172257,  15.77690432,
         5.55479442,  -6.29699128,  -1.24561062,  16.85002635,
        47.63536812,  15.71002778])

In [ ]:
md_sim.optimize_confs(mol)
md_sim.get_conformer_energies(mol)

In [ ]:
nonring, ring = TorsionFingerprints.CalculateTorsionLists(mol)
nonring = [list(atoms[0]) for atoms, ang in nonring]

random_energies = []
for trial in range(10):
    embedded = Chem.Mol(mol)
    embedded.RemoveAllConformers()
    conf = mol.GetConformer(0)

    action = np.random.randint(0, 6, len(nonring))
    for idx, tors in enumerate(nonring):
        deg = Chem.rdMolTransforms.GetDihedralDeg(conf, *tors)
        ang = -180.0 + 60 * action[idx]
        Chem.rdMolTransforms.SetDihedralDeg(conf, tors[0], tors[1], tors[2], tors[3], float(ang))
    embedded.AddConformer(conf, assignId=True)
    md_sim.optimize_confs(embedded)
    energy = md_sim.get_conformer_energies(embedded)
    print(energy)
    random_energies.append(energy)
print(random_energies)

In [4]:
def np_to_mm(arr: np.ndarray, unit: openmm.unit=u.angstrom):
    wrapped_val = openmm.unit.quantity.Quantity(arr, unit)
    return wrapped_val

In [ ]:
pos_a = np_to_mm(mol.GetConformer(0).GetPositions())
pos_b = ala5_crds.positions

In [5]:
import sys

# OpenMM Imports
import simtk.openmm as mm
import simtk.openmm.app as app

# ParmEd Imports
from parmed.charmm import CharmmPsfFile, CharmmCrdFile, CharmmParameterSet
from parmed.openmm import StateDataReporter
from parmed import unit as u

# Load the CHARMM files
print('Loading CHARMM files...')
toppar = [
    "toppar/par_all36_prot.prm", 
    "toppar/top_all36_prot.rtf",
    "toppar/toppar_water_ions.str",
]

params = CharmmParameterSet(*toppar)
ala5_gas = CharmmPsfFile('1uao_8_autopsf.psf')

system = ala5_gas.createSystem(params, nonbondedMethod=app.NoCutoff,
                               constraints=app.HBonds, implicitSolvent=app.HCT,
                               implicitSolventSaltConc=0.1 * u.moles/u.liter
)

# Create the integrator to do Langevin dynamics
integrator = mm.LangevinIntegrator(
                        300 * u.kelvin,       # Temperature of heat bath
                        1.0 / u.picoseconds,  # Friction coefficient
                        2.0 * u.femtoseconds, # Time step
)

# Define the platform to use; CUDA, OpenCL, CPU, or Reference. Or do not specify
# the platform to use the default (fastest) platform
platform = mm.Platform.getPlatformByName('CUDA')
prop = dict(CudaPrecision='mixed') # Use mixed single/double precision

# Create the Simulation object
sim = app.Simulation(ala5_gas.topology, system, integrator, platform, prop)

/home/yppatel/anaconda3/envs/myenv/lib/python3.8/site-packages/parmed/charmm/parameters.py:880: UserWarning: WARNING: Ignoring "DELETE ACCE NE2" because entity type ACCE not used.
  warnings.warn('WARNING: Ignoring "%s" because entity type %s not '


Loading CHARMM files...


In [6]:
ala5_crds = app.PDBFile('1uao_12_autopsf.pdb')

sim.context.setPositions(ala5_crds.positions)
energy = sim.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(u.kilocalories_per_mole)    
print(f"before: {energy}")
sim.minimizeEnergy(maxIterations=2000)
energy = sim.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(u.kilocalories_per_mole)    
print(f"after: {energy}")

before: 32.464435790823984 kcal/mol
after: -237.72226764762001 kcal/mol


In [10]:
pdb = "/home/yppatel/misc/clean_idp_rl/disordered_mol_untrained/mol0.pdb"
mol = Chem.rdmolfiles.MolFromPDBFile(pdb, removeHs=False)
AllChem.EmbedMultipleConfs(mol, numConfs=1000, numThreads=-1)

[18:18:52] Molecule does not have explicit Hs. Consider calling AddHs()


In [12]:
from rdkit.Geometry import Point3D

In [13]:
energies = []
for i in range(mol.GetNumConformers()):
    pos_a = np_to_mm(mol.GetConformer(i).GetPositions())
    sim.context.setPositions(pos_a)
    try:
        sim.minimizeEnergy(maxIterations=1000)
    except:
        energies.append(float("inf"))
        continue

    optimized_positions_nm = sim.context.getState(getPositions=True).getPositions()
    optimized_positions = optimized_positions_nm.in_units_of(u.angstrom) # match RDKit/MMFF convention

    conf = mol.GetConformer(i)
    for j, pos in enumerate(optimized_positions):
        conf.SetAtomPosition(j, Point3D(pos.x, pos.y, pos.z))

    energy = sim.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(u.kilocalories_per_mole)
    energies.append(energy)

In [17]:
clean_energies = [energy._value if not isinstance(energy, float) else energy for energy in energies]

In [18]:
sort = np.argsort(clean_energies)  # sort by increasing energy

new = Chem.Mol(mol)
new.RemoveAllConformers()

for i in sort:
    conf = mol.GetConformer(int(i))
    new.AddConformer(conf, assignId=True)

In [21]:
pos_a = np_to_mm(new.GetConformer(4).GetPositions())
sim.context.setPositions(pos_a)
sim.minimizeEnergy(maxIterations=1000)
sim.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(u.kilocalories_per_mole)

Quantity(value=-235.45845195530066, unit=kilocalorie/mole)

In [ ]:
Chem.rdmolfiles.MolToPDBFile(new, "new.pdb")

In [193]:
ala5_crds = app.PDBFile('1uao_0_autopsf.pdb')

sim.context.setPositions(ala5_crds.positions)
energy = sim.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(u.kilocalories_per_mole)    
print(f"before: {energy}")
sim.minimizeEnergy(maxIterations=2000)
energy = sim.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(u.kilocalories_per_mole)    
print(f"after: {energy}")

pdb = "/home/yppatel/misc/clean_idp_rl/disordered_mol_untrained/mol0.pdb"
mol = Chem.rdmolfiles.MolFromPDBFile(pdb, removeHs=False)

for i in range(mol.GetNumConformers()):
    pos_a = np_to_mm(mol.GetConformer(i).GetPositions())
    sim.context.setPositions(pos_a)
    try:
        sim.minimizeEnergy(maxIterations=1000)
        energy = sim.context.getState(getEnergy=True).getPotentialEnergy().in_units_of(u.kilocalories_per_mole)
    except:
        energy = float("inf")
    print(f"{i} -> {energy}")

before: 72.3340359596176 kcal/mol
after: -241.45202879668082 kcal/mol
0 -> -207.01181990946753 kcal/mol
1 -> -223.0982794259196 kcal/mol
2 -> -164.08587823480775 kcal/mol
3 -> -205.5200195508545 kcal/mol
4 -> -223.31813780042162 kcal/mol
5 -> -209.4291055020327 kcal/mol
6 -> -203.74791086448494 kcal/mol
7 -> -202.89361208254903 kcal/mol
8 -> inf
9 -> -170.54804199125653 kcal/mol
10 -> -212.60953687601634 kcal/mol
11 -> -216.54962973537002 kcal/mol
12 -> inf
13 -> inf
14 -> -221.50640174749284 kcal/mol
15 -> -112.90965918991996 kcal/mol
16 -> -215.70211926531195 kcal/mol
17 -> -212.2574957772419 kcal/mol
18 -> -201.15626722768945 kcal/mol
19 -> -212.88529359965437 kcal/mol
20 -> -209.49068272077713 kcal/mol
21 -> -202.5351923991402 kcal/mol
22 -> -193.40544204883008 kcal/mol
23 -> -148.47447990694005 kcal/mol
24 -> -181.76165930925515 kcal/mol
25 -> -205.26167160401098 kcal/mol
26 -> -191.38253955515515 kcal/mol
27 -> -184.96786013807193 kcal/mol
28 -> -219.3909777433221 kcal/mol
29 -> 

In [192]:
pos_a

Quantity(value=array([[-3.0784e+01, -5.5152e+01, -4.2550e+00],
       [-3.1284e+01, -5.5908e+01, -4.7670e+00],
       [-2.9868e+01, -5.5529e+01, -3.9370e+00],
       [-3.0590e+01, -5.4387e+01, -4.9320e+00],
       [-3.1656e+01, -5.4673e+01, -3.1300e+00],
       [-3.1812e+01, -5.5510e+01, -2.4650e+00],
       [-3.2578e+01, -5.4326e+01, -3.5710e+00],
       [-3.1106e+01, -5.3499e+01, -2.3640e+00],
       [-3.0793e+01, -5.2474e+01, -2.9580e+00],
       [-3.0966e+01, -5.3529e+01, -1.0200e+00],
       [-3.0485e+01, -5.2735e+01, -6.5700e-01],
       [-3.1495e+01, -5.4482e+01, -4.6000e-02],
       [-3.1216e+01, -5.4070e+01,  9.1200e-01],
       [-3.0792e+01, -5.5831e+01, -1.5000e-02],
       [-3.1302e+01, -5.6812e+01,  5.1400e-01],
       [-3.3052e+01, -5.4589e+01, -3.3000e-02],
       [-3.3388e+01, -5.5483e+01,  5.3400e-01],
       [-3.3469e+01, -5.4677e+01, -1.0590e+00],
       [-3.3651e+01, -5.3379e+01,  6.3600e-01],
       [-3.4029e+01, -5.3445e+01,  1.9890e+00],
       [-3.3874e+01, -5.4